In [1]:
import tensorflow as tf

In [2]:
!sudo apt-get install libtinfo5
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  accountsservice-ubuntu-schemas bc bluez-obexd cups cups-browsed cups-client
  cups-common cups-core-drivers cups-daemon cups-filters
  cups-filters-core-drivers cups-ipp-utils cups-ppdc cups-server-common
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript
  gir1.2-dbusmenu-glib-0.4 gnome-bluetooth gnome-power-manager
  gnome-screensaver gsettings-ubuntu-schemas gvfs-backends indicator-applet
  indicator-application indicator-appmenu indicator-bluetooth indicator-common
  indicator-datetime indicator-keyboard indicator-messages indicator-power
  indicator-printers indicator-session indicator-sound jayatana
  libaccounts-glib0 libbamf3-2 libcdio-cdda2 libcdio-paranoia2 libcdio18
  libcupsfilters1 libfcitx-config4 libfcitx-gclient1 libfcitx-utils0
  libfontembed1 libgnome-panel0 libgs9 libgs9-comm

In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [4]:
#train_data = pd.read_csv('data/advanced_general_conversation_test_B.csv')
#train_data = pd.read_csv('advanced_general_conversation_test_B.csv')
train_data = pd.read_csv('advanced_general_conversation_testB_enlarged.csv')
test_data = pd.read_csv('test.csv')

In [5]:
train_data

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
5528,5528,일반 대화,진짜 오늘 날씨 너무 덥지 않니?\n 맞아 진짜 더워. 오늘 밖에 나가기가 싫어....
5529,5529,일반 대화,진짜 오늘 날씨 너무 좋지 않니?\n 그래 정말 좋아. 우리 오늘 주말에 뭐할까?...
5530,5530,일반 대화,제발 내 말 좀 들어봐. 지금 우리 진짜 심각한 상황에 처했어.\n 무슨 일인데 ...
5531,5531,일반 대화,진짜 지금 당장 우리 그냥 만나서 얘기 좀 해보자.\n 무슨 일인데 그래?\n 관...


In [6]:
train_data['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '일반 대화'],
      dtype=object)

In [7]:
label = {'협박 대화' : 0, '갈취 대화' : 1, '직장 내 괴롭힘 대화' :2, '기타 괴롭힘 대화' : 3, '일반 대화' : 4}

In [8]:
train_data['int_class'] = train_data['class'].map(label)

In [9]:
import re
train_data['conversation'] = train_data['conversation'].apply(lambda x: re.sub(r'[\n…]', '', x))

In [10]:
train_data

,idx,class,conversation,int_class
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼...,0
1,1,협박 대화,길동경찰서입니다.9시 40분 마트에 폭발물을 설치할거다.네?똑바로 들어 한번만 더 ...,0
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.그만해. 니들 놀리는거 재미없어...,3
3,3,갈취 대화,어이 거기예??너 말이야 너. 이리 오라고무슨 일.너 옷 좋아보인다?얘 돈 좀 있나...,1
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1
...,...,...,...,...
5528,5528,일반 대화,진짜 오늘 날씨 너무 덥지 않니? 맞아 진짜 더워. 오늘 밖에 나가기가 싫어. 제...,4
5529,5529,일반 대화,진짜 오늘 날씨 너무 좋지 않니? 그래 정말 좋아. 우리 오늘 주말에 뭐할까? 나...,4
5530,5530,일반 대화,제발 내 말 좀 들어봐. 지금 우리 진짜 심각한 상황에 처했어. 무슨 일인데 그래...,4
5531,5531,일반 대화,진짜 지금 당장 우리 그냥 만나서 얘기 좀 해보자. 무슨 일인데 그래? 관계 때문...,4


In [11]:
from transformers import BertTokenizer, TFBertModel

In [12]:
#MODEL_NAME = "monologg/koelectra-small-v2-distilled-korquad-384"
#tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME, num_labels=5, from_pt=True)

In [13]:
MODEL_NAME = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = TFBertModel.from_pretrained(MODEL_NAME, num_labels=5, from_pt=True)

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [14]:
print(tokenizer.vocab_size)


30000


In [15]:
# add_tokens = ['[NEW']
new_words = ['구급차', '지혈', '침입', '고속도로', '엘리베이터','진정', '한강공원', '빌라', '소방차','대피', '편의점', '부엌', '심폐소생술', '침착', '출혈', '빌딩', '중학교', '전여친', '전남친', '안부', '지각', '팀장', '##팀장', '팀원', '##팀원', '스캔들', '읽씹', '간식', '프로젝트', '믹스커피', '군것질', '필수품', '힐링', '시발', '씨발', '상사', '피드백', '성의', '레스토랑', '칼퇴', '서운', '퇴사','숙제', '농구', '반장', '쌍욕', '망각', '수다', '떨기', '음료수', '벌점', '지각', '교복', 'PPT', '타이트', '미팅', '희정', '영지', '네일', '파스텔', '자릿세', '보이스피싱', '아이패드', '뺀질', '휠체어', '옥상', '노약자', '##석', '미스##', '데이트', '민아', '말년', '##과장', '과장', '민서', '##씨', '경마장', '숙제', '##대리', '대리', '지수', '교무실', '##주임',
'주임', '진희', '병신', '차장',  '##차장', '사원', '##사원', '이##', '김##', '박##', '가평', '강남', '거창', '관악', '구로', '나주', '대구', '대만', '대전', '독도', '독일', '동해', '로마', '몽골', '미국', '부산', '부여', '북촌', '북한','강민', '관우', '광수', '나은', '민재', '민정', '민지', '민철', '민하', '민호', '민희', '개독', '꼰대', '기레기','구글', '구찌', '농협', '던힐', '레종', '맥북', '샤넬', '힐튼', '나이키', '넥서스', '롤렉스', '리니지', '말보로', '빼빼로', '생로랑', '샤오미']

In [16]:
new_words = list(set(new_words))
len(new_words)

140

In [17]:
for item in new_words:
    if len(tokenizer.encode(item))==3:
        print(item, tokenizer.encode(item))
        new_words.remove(item)
len(new_words)


대리 [2, 14685, 3]
편의점 [2, 15858, 3]
꼰대 [2, 11293, 3]
독도 [2, 12037, 3]
부여 [2, 20697, 3]
나은 [2, 14112, 3]
북한 [2, 8018, 3]
과장 [2, 21002, 3]
고속도로 [2, 19340, 3]
대전 [2, 20532, 3]
기레기 [2, 8500, 3]
구글 [2, 16910, 3]
대만 [2, 12698, 3]
망각 [2, 29908, 3]
동해 [2, 20875, 3]
쌍욕 [2, 17589, 3]
개독 [2, 13384, 3]
민정 [2, 12077, 3]
대구 [2, 8303, 3]
강남 [2, 9134, 3]
시발 [2, 13552, 3]
부산 [2, 9530, 3]
독일 [2, 11766, 3]


117

In [18]:
num_added_toks = tokenizer.add_tokens(new_words)
print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model.resize_token_embeddings(tokenizer.vocab_size+num_added_toks)

We have added 110 tokens


In [19]:
print("Model vocab size:", model.config.vocab_size)

Model vocab size: 30110


In [20]:
model.config.vocab_size

30110

In [21]:
print(model.summary())

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109003008 
Total params: 109,003,008
Trainable params: 109,003,008
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# 테스트 데이터 토큰화 (문장별 토큰 개수 확인)
tokenized_texts = [tokenizer.tokenize(text) for text in train_data['conversation']]

# 각 문장의 토큰 개수 계산
token_lengths = [len(tokens) for tokens in tokenized_texts]

# 최대 토큰 개수 찾기
max_token_length = max(token_lengths)

print(f"문장에서 가장 긴 토큰 개수: {max_token_length}")

문장에서 가장 긴 토큰 개수: 599


In [23]:
# 단어 사전 개수 확인
print(tokenizer.vocab_size) # orig size
print(len(tokenizer)) # added size


30000
30110


In [24]:
# 특수 토큰 확인
print(tokenizer.cls_token)
print(tokenizer.sep_token)

[CLS]
[SEP]


In [25]:
count = 0
for i in range(len(tokenized_texts)):
    if token_lengths[i]>300:
        print(count)
        print(tokenized_texts[i])
        print('\n')
        count += 1

0
['야', '신규', '너', '똑바로', '환자', '관리', '안', '하냐', '?', '인수', '##인', '##계', '제대로', '들은', '##거', '맞아', '?', '너', '누가', '저', '환자', '항', '##암', '약', '##물', '속도', '저렇게', '하', '##래', '죽이고', '싶어서', '환장', '##했어', '?', '인수', '##인', '##계', '들었는데', '.', '어', '.', '약', '##물', '속도', '##는', '제', '프리', '##셉', '##터', '선생님', '##이', '.', '설명', '##해주', '##신', '##바로', '.', '저렇게', '하라고', '하셨', '##어요', '.', '니', '프리', '##셉', '##터', '##가', '문제', '##라는거야', '?', '너가', '잘', '못', '##들은', '##거', '아니고', '?', '아니', '바쁘', '##셔서', '그런지', '.', '책', '보고', '알아서', '하라', '그래서', '책', '보고', '최대한', '아는', '대로', '한', '##겁니다', '.', '모르는', '##거', '물어보', '##라고', '프리', '##셉', '##터', '붙여', '##준거', '아니야', '?', '너', '여기', '실', '##전이', '##야', '니', '하는', '행동에', '환자가', '죽을', '##수도', '있다고', '.', '야', '김', '##간', '##호사', '너', '얘', '프리', '##셉', '##터', '##지', '?', '네', '맞는데', '##요', '왜', '그러', '##시죠', '?', '아니', '얘가', '약', '##물', '조절', '잘못', '##했는데', '너가', '책', '보고', '하라', '그랬다', '##고', '그러', '##던데', '?', '대충', '알려', '##줬', '##냐', '?', '그런',

In [26]:
# 최대 토큰 인텍스, 문장, 토큰 수 확인
max_token_idx = token_lengths.index(max(token_lengths))  # 가장 긴 문장의 인덱스
max_token_text = train_data['conversation'].iloc[max_token_idx]  # 해당 문장
max_token_count = token_lengths[max_token_idx]

In [27]:
max_token_text

' 그래 오늘 날씨가 진짜 좋네. 어때 우리 산책할까? 좋은 생각인 것 같아. 그런데 지금 시간이 어때? 지금 딱 좋아. 한 시간 정도 걸리면 충분할 거야. 좋아 가자. 그런데 우리가 그냥 아무데나 가면 어떡해? 아니야 나는 미리 검색해뒀어. 좋은 산책로가 있대. 그래 그럼 나도 가볼게. 그런데 제발 너무 멀리 가지 말자. 그래 걱정 마. 너무 멀지 않은 곳이니까. 좋아 그럼 지금 바로 갈게. 그런데 내가 오늘 교수님 연구에 참여하느냐 안 하느냐에 따라 시간이 달라질 수 있어. 아니 그럴 필요 없어. 교수님은 네가 오늘 연구에 참여하지 않아도 괜찮다고 하셨어. 진짜? 그럼 다행이야. 그럼 지금 바로 갈게. 좋아 그럼 만나자. 제발 지금 바로 그 문제를 해결해야 해. 너무 늦어지면 안 돼. 그래 하지만 내가 지금 바빠서 바로 처리할 수 없어. 아니야 이건 정말 중요한 문제야. 내가 도와줄까? 아니 괜찮아. 내가 알아서 처리할게. 하지만 진짜 고마워. 그래 그럼 내가 도와줄게. 내가 지금 바로 그 문제를 해결할게. 아니야 정말 고마워. 하지만 내가 해결하는 게 더 좋아. 좋아 그럼 내가 기다릴게. 하지만 너무 늦어지면 안 돼. 그래 알았어. 최대한 빨리 해결하도록 노력할게. 좋아 그럼 지금 바로 다른 일을 처리할게. 그래 고마워. 그 사람 진짜 별로야. 오늘 나한테 너무 무례하게 굴었어. 그래 나도 그 사람한테 좀 실망했어. 아니야 나는 그 사람한테 너무 화가 나. 그래 나도 그 사람한테 실망했어. 하지만 너무 화를 내지는 말자. 아니야 나는 그 사람한테 진짜 사과를 받고 싶어. 그래 나도 그 사람한테 실망했어. 하지만 너무 화를 내지는 말자. 좋아 그럼 내가 그 사람에게 사과를 요구할게. 그래 좋아. 하지만 너무 화를 내지는 말자. 그래 알았어. 내가 그 사람에게 사과를 요구할게. 좋아 그럼 지금 바로 처리하자. 우리 오늘 데이트 어때? 그래 좋은 생각인 것 같아. 하지만 내가 지금 바빠서 어떡해? 아니야 나는 미리 예약해뒀어. 오늘 저녁에 맛있는 음식이 기다리

In [28]:
max_token_count

599

In [29]:
#MAX_LEN = max_token_count + 2 # 문장 최대 토큰 길이
MAX_LEN = 300 # max length of bert model
NUM_LABELS = 5  # 분류할 클래스 개수
BATCH_SIZE = 8
EPOCHS = 1

In [30]:
# 텍스트를 토큰화하는 함수
def tokenize_texts(texts, labels):
    encodings = tokenizer(
        list(texts),  # 리스트 형태로 변환
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN, 
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"], tf.convert_to_tensor(labels, dtype=tf.int32)

In [31]:
# 데이터 분할 (8:2 비율로 train/val 나누기)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['conversation'], train_data['int_class'], test_size=0.2, random_state=42
)

In [32]:
# 데이터 토큰화
train_input_ids, train_attention_mask, train_labels = tokenize_texts(train_texts, train_labels)
val_input_ids, val_attention_mask, val_labels = tokenize_texts(val_texts, val_labels)

In [33]:
# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": train_input_ids, "attention_mask": train_attention_mask}, train_labels)
).shuffle(len(train_texts)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": val_input_ids, "attention_mask": val_attention_mask}, val_labels)
).batch(BATCH_SIZE)

In [34]:
# 데이터 변환 (train_data['conversation'], train_data['int_class'] 사용)
input_ids, attention_mask, label_tensor = tokenize_texts(train_data['conversation'], train_data['int_class'])

# TensorFlow Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": input_ids, "attention_mask": attention_mask}, label_tensor)
)

# 배치 및 섞기
dataset = dataset.shuffle(len(train_data)).batch(BATCH_SIZE)

In [35]:
#from tensorflow.keras import backend as K

In [36]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=1)  # 가장 높은 확률값을 가진 클래스 선택
        self.precision.update_state(y_true, y_pred)
        self.recall.update_state(y_true, y_pred)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))  # F1-score 계산

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [37]:
input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

# BERT 모델에 입력
bert_output = model(input_ids, attention_mask=attention_mask)[1]
x = Dropout(0.5)(bert_output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_LABELS, activation="softmax")(x)  # 다중 클래스 분류

# 모델 정의
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

# 모델 구조 확인
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 300)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 300)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109003008   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 768)          0           tf_bert_model[0][1]          

In [38]:
# EarlyStopping 콜백 설정 (patience=2)
early_stopping = EarlyStopping(
    monitor="val_loss",  # 검증 데이터 손실 기준
    patience=3,  # 3epoch 동안 개선되지 않으면 종료
    restore_best_weights=True  # 가장 성능이 좋았던 가중치 복원
)

In [39]:
# ModelCheckpoint 설정 (최적 모델 저장)
model_checkpoint = ModelCheckpoint(
    "best_kobert_testB_ep1_dr04.h5",  # 저장할 모델 파일명
    monitor="val_loss",  # 검증 손실 기준으로 저장
    save_best_only=True,  # 가장 성능 좋은 모델만 저장
    save_weights_only=True,  # 가중치만 저장 (전체 모델 저장하려면 False)
    verbose=1  # 저장될 때 로그 출력
)

In [40]:
#print(early_stopping)  # 정상적으로 설정되었는지 확인
#print(model_checkpoint)

In [41]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[model_checkpoint]
)

Epoch 1/2
554/554 [==============================] - 325s 558ms/step - loss: 0.6542 - accuracy: 0.7677 - val_loss: 0.2829 - val_accuracy: 0.9051

Epoch 00001: val_loss improved from inf to 0.28295, saving model to best_kobert_testB_ep1_dr04.h5
Epoch 2/2
554/554 [==============================] - 310s 559ms/step - loss: 0.2540 - accuracy: 0.9261 - val_loss: 0.2868 - val_accuracy: 0.9160

Epoch 00002: val_loss did not improve from 0.28295


In [58]:
model = TFBertModel.from_pretrained(model_checkpoint, local_files_only=True)

Cannot find the requested files in the cached path and outgoing traffic has been disabled. To enable model look-ups and downloads online, set 'local_files_only' to False.


OSError: Can't load config for '<keras.callbacks.ModelCheckpoint object at 0x7995130d0a60>'. Make sure that:

- '<keras.callbacks.ModelCheckpoint object at 0x7995130d0a60>' is a correct model identifier listed on 'https://huggingface.co/models'

- or '<keras.callbacks.ModelCheckpoint object at 0x7995130d0a60>' is the correct path to a directory containing a config.json file



In [42]:
from sklearn.metrics import f1_score

In [43]:
# 검증 데이터 예측
val_predictions = model.predict({"input_ids": val_input_ids, "attention_mask": val_attention_mask})
predicted_classes = np.argmax(val_predictions, axis=1)  # 확률값을 가장 높은 클래스 인덱스로 변환

# F1-score 계산
f1 = f1_score(val_labels, predicted_classes, average="weighted")

print(f"검증 데이터 F1-score: {f1:.4f}")

검증 데이터 F1-score: 0.9161


In [44]:
def tokenize_test_texts(texts):
    encodings = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"]

In [45]:
test_input_ids, test_attention_mask = tokenize_test_texts(test_data['text'])

In [46]:
# 모델 예측 수행
predictions = model.predict({"input_ids": test_input_ids, "attention_mask": test_attention_mask})

# 확률값을 가장 높은 클래스 인덱스로 변환
predicted_classes = np.argmax(predictions, axis=1)

# 결과를 데이터프레임에 추가
test_data['target'] = predicted_classes

In [47]:
# 예측된 결과 출력
print(test_data[['text', 'target']])

                                                  text  target
0    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...       1
1    우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...       2
2    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...       2
3    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...       4
4    아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...       3
..                                                 ...     ...
495  미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...       2
496  교수님 제 논문에 제 이름이 없나요?  아 무슨 논문말이야?  지난 번 냈던 논문이...       1
497  야 너  네 저요? 그래 너 왜요 돈좀 줘봐  돈 없어요 돈이 왜 없어 지갑은 폼이...       1
498  야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...       2
499  엄마 저 그 돈 안해주시면 정말 큰일나요.  이유도 말하지 않고. 몇번째니 경민아....       0

[500 rows x 2 columns]


In [48]:
test_data

,idx,text,target
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,1
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,2
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,2
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,4
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,3
...,...,...,...
495,t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,2
496,t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,1
497,t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,1
498,t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,2


In [49]:
# CSV로 저장
test_data[['idx', 'target']].to_csv("submission_kobert_testB_ep2_preprocessing_dr04.csv", index=False)

In [50]:
test_data['target'].value_counts()

3    158
1    115
2    115
0    102
4     10
Name: target, dtype: int64

In [51]:
test_data[test_data['target'] == 4]

,idx,text,target
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,4
57,t_057,아니 진짜 걍 지구 망할 거 같아 위쪽은 비 난리고 밑에는 비는 안 와도 공기 난...,4
82,t_082,"천안 두정동 제이빌라 인데요, 지금 화재가 발생했어요. 빌라 어디에 불이 났나요? ...",4
152,t_152,"시신을 발견했습니다. 위치가 어디시죠? 인천 영흥도인근 해상인데요, 검은색 옷을 입...",4
162,t_162,회사에서 성희롱 타깃이 된 것 같아서 우울해. 직장 내 성희롱 때문에 우울하시군요....,4
221,t_221,"좋은 아침입니다. 이동진 대표님. 좋은 아침. 밖에 비가 꽤 많이 오던데, 출근하시...",4
244,t_244,토요일 오빠혼자해야하자나 구럼 금요일 저녁에 오것지 엄마 일단 금요일은 내가해욤 그...,4
468,t_468,시험에 자신이 없어서 일부러 아픈 척을 하고 학교를 안 갔어. 해방감이 들었어. 시...,4
487,t_487,저녁 먹었어 아니 오늘 뭐먹을까 오늘 집에서 라면 먹자 무슨 라면 먹을건데 열라면에...,4
491,t_491,"양재 터널 입구 앞인데요. 사고가 났거든요? 위치가 양재 터널 맞으시죠? 네, 지금...",4


In [52]:
# 긴 텍스트 출력 제한 해제
pd.options.display.max_colwidth = None  
# 예측된 결과 출력
display(test_data[test_data['target'] == 4])

,idx,text,target
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래,4
57,t_057,아니 진짜 걍 지구 망할 거 같아 위쪽은 비 난리고 밑에는 비는 안 와도 공기 난리고 이제 또 비오는 너무 많이와 진짜 망할려고이러나 그러게 말이여 진짜 몇년 전부터 미세먼지도 난리고 원래는 봄에만 황사처럼 공기 안 좋았잖아 근데 요즘엔 걍 사계절 내내 안 좋으니까 약간 더 무서운 건 또 거기에 다 적응한 거임 일단 나부터 적응 다함 공기 안 좋은 거 그러려니해 아 개인정 어쩌다 이렇게 됐는지 진짜 예전이 그립다 초딩 때라던가 그냥 벌받는거 같기두 하구 마잡 그래서 나 늦은 감 있지만 이제라도 일회용 줄이려고 뭔가 이젠 다들 좀 경각심 느껴야 할 거 같음\n,4
82,t_082,"천안 두정동 제이빌라 인데요, 지금 화재가 발생했어요. 빌라 어디에 불이 났나요? 1층 주차장에서 불이 나고 있어요. 그러면 지금 다친 사람도 있나요? 아직까지는 없는거 같아요. 알겠습니다. 지금 119 소방대를 보내도록 하겠습니다.",4
152,t_152,"시신을 발견했습니다. 위치가 어디시죠? 인천 영흥도인근 해상인데요, 검은색 옷을 입은 남자가 떠있는것을 발견했어요. 그럼 가까운 지상으로 시신을 가져와주실수 있나요? 네 영흥도 부둣가로 갈게요 알겠습니다.",4
162,t_162,회사에서 성희롱 타깃이 된 것 같아서 우울해. 직장 내 성희롱 때문에 우울하시군요. 좀 더 이야기 해 주시겠어요? 다른 직원들이 옆자리 직원을 성희롱 하는 게 보여서 도와줬더니 이번엔 내가 타깃이 되었어. 어떻게 하면 성희롱 문제를 해결할 수 있을까요? 내일 직장 내 성희롱 관련 담당자를 찾아가봐야겠어. 일이 잘 풀려서 우울한 기분이 나아지면 좋겠어요.,4
221,t_221,"좋은 아침입니다. 이동진 대표님. 좋은 아침. 밖에 비가 꽤 많이 오던데, 출근하시는데 고생 많이 하셨을 거 같습니다. 그래도 무사히 왔잖아. 괜찮아. 다행입니다. 회의는 언제 시작해? 곧 회의 시작할 것 같습니다. 좋은 아침입니다. 이동진 대표님. 좋은 아침. 밖에 비가 꽤 많이 오던데, 출근하시는데 고생 많이 하셨을 거 같습니다. 그래도 무사히 왔잖아. 괜찮아. 다행입니다. 회의는 언제 시작해? 곧 회의 시작할 것 같습니다.",4
244,t_244,토요일 오빠혼자해야하자나 구럼 금요일 저녁에 오것지 엄마 일단 금요일은 내가해욤 그래 오늘 내일 아무도 없음 금요일 머가 아무도없덩 사람 집에 엥 그오빠도 다 감 시골 뭐 오빤 더 좋은거지\n,4
468,t_468,시험에 자신이 없어서 일부러 아픈 척을 하고 학교를 안 갔어. 해방감이 들었어. 시험을 안 봐서 기분이 좋으시군요. 지금까지 시험이 나를 얼마나 옭아맸는지 알게 돼서 한국의 시험 문화에 화가 나. 화나는 감정이 나아지려면 어떻게 해야 할까요? 한국 시험 문화에 대한 분노를 시험을 대하는 즐거움으로 변화시켜야겠어. 목표대로 시험을 즐겁게 느낄 수 있게 되길 바라요.,4
487,t_487,저녁 먹었어 아니 오늘 뭐먹을까 오늘 집에서 라면 먹자 무슨 라면 먹을건데 열라면에 순두부 넣어먹는게 맛있대 진짜 맛있어 내 친구가 꼭 먹으래 와 그러면 오늘 꼭 먹어보자 내가 순두부랑 열라면 사갈게 그래그래 오늘 늦지 말고 들어와 응 알았어,4
491,t_491,"양재 터널 입구 앞인데요. 사고가 났거든요? 위치가 양재 터널 맞으시죠? 네, 지금 버스가 승용차 한대를 덮쳤어요. 버스가 승용차를 덮쳤다고요 지금 사고가 일어난건가요? 네 운전자 분은 괜찮으세요? 워낙 사고가 크게나서 잘 모르겠어요. 알겠습니다.",4


In [60]:
pd = pd.read_csv('submission_kobert_testB_ep1_preprocessing.csv')


In [61]:
pd['target'].value_counts()

0    136
3    124
2    116
1     99
4     25
Name: target, dtype: int64